In [ ]:
from gs_quant.session import GsSession, Environment
client_id = None # Supply your application id
client_secret = None # Supply your client secret
scopes = ('run_analytics',)

GsSession.use(Environment.PROD, client_id, client_secret, scopes)

In [ ]:
from gs_quant.markets import PricingContext
from gs_quant.markets.portfolio import Portfolio
from gs_quant.instrument import IRSwap
from gs_quant.risk import CarryScenario, IRFwdRate
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# construct a portfolio of swaps with tenors 6m apart and resolve these trades

%%time
tenor=['42m','4y','54m','5y']
swap_port = [IRSwap('Pay', t, 'EUR', fixed_rate='ATM', floating_rate_frequency='6m', effective_date='6m',name='EUR6m'+t) for t in tenor]
port = Portfolio(swap_port)
port.resolve()

fwd_results = []
spot_results = []

# calculate the fwd rate under the assumption spot rates are held constant and that fwd rates are held constant.
# here we calculate the fwd rate every week forward in time.

with PricingContext():
    for time_shift in range(0,120,5):
        with CarryScenario(time_shift, roll_to_fwds=True):
            fwd_results.append(port.calc(IRFwdRate))
        with CarryScenario(time_shift, roll_to_fwds=False):
            spot_results.append(port.calc(IRFwdRate))



In [ ]:
# graph the results.  Note that under the fwd rate constant assumption the fwd rate doesn't change, whereas you see the effect
# of the rolldown of the curve in the spot rate constant assumption.

for swap in swap_port:
    pd.Series(p[swap] for p in spot_results).plot(figsize=(20,12), title='Swap Carry',label='{} spot curve carry'.format(swap.name))
    pd.Series(p[swap] for p in fwd_results).plot(label='{} fwd curve carry'.format(swap.name))

plt.xlabel('TimeShift (in weeks)')
plt.ylabel('Rate')
plt.legend()